In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import dask
import dask.dataframe as dd
import time
import os
from IPython.display import display
import ast

# import sweetviz as sv


# print(dask.__version__)

In [37]:
# Convert JSON to parquet files

# Paths to save the Parquet files
# parquet_dir = "../../data/processed/"

# # 1. 
# # Convert pandas DataFrames to Parquet
# data_tip = pd.read_json(f"../../data/raw/yelp_academic_dataset_tip.json", lines=True)
# data_tip.to_parquet(os.path.join(parquet_dir, "yelp_academic_dataset_tip.parquet"))

# # Convert Dask DataFrames to Parquet
# data_businesses = dd.read_json(f"../../data/raw/yelp_academic_dataset_business.json", lines=True)
# data_businesses.to_parquet(os.path.join(parquet_dir, "yelp_academic_dataset_business.parquet"))

# data_checkin = dd.read_json(f"../../data/raw/yelp_academic_dataset_checkin.json", lines=True)
# data_checkin.to_parquet(os.path.join(parquet_dir, "yelp_academic_dataset_checkin.parquet"))

# data_user = dd.read_json(f"../../data/raw/yelp_academic_dataset_user.json", lines=True)
# data_user.to_parquet(os.path.join(parquet_dir, "yelp_academic_dataset_user.parquet"))

# # Read the large JSON file with Dask in chunks and convert to Parquet
# data_review = dd.read_json(f"../../data/raw/yelp_academic_dataset_review.json", lines=True, blocksize="64MB")
# data_review.to_parquet(os.path.join(parquet_dir, "yelp_academic_dataset_review.parquet"))



# Read and save sample data for larger files
# data_user = dd.read_json('../../data/raw/yelp_academic_dataset_user.json', lines=True, blocksize=40000000)  # 1MB blocksize
# data_user_sample = data_user.head(5000)  # Adjust the number of rows to read
# data_user_sample.to_parquet(os.path.join(parquet_dir, "yelp_academic_dataset_review_user.parquet"))

# # Review Data
# data_review = dd.read_json(f"../../data/raw/yelp_academic_dataset_review.json", lines=True, blocksize=40000000)
# data_review = data_review.head(5000)
# data_review.to_parquet(os.path.join(parquet_dir, "yelp_academic_dataset_review_sample.parquet"))











In [18]:

# Load data and set indices
data_businesses = dd.read_parquet("../../data/processed/yelp_academic_dataset_business.parquet")
data_businesses = data_businesses.set_index('business_id')

data_checkin = dd.read_parquet("../../data/processed/yelp_academic_dataset_checkin.parquet")
data_checkin = data_checkin.set_index('business_id')

data_tip = dd.read_parquet("../../data/processed/yelp_academic_dataset_tip.parquet")

data_review_sample = dd.read_parquet("../../data/processed/yelp_academic_dataset_review_sample.parquet")
data_review_sample = data_review_sample.set_index('review_id')

data_user_sample = dd.read_parquet("../../data/processed/yelp_academic_dataset_user_sample.parquet")
data_user_sample = data_user_sample.set_index('user_id')

# Ensure index names are set correctly
# data_tip = data_tip.reset_index(drop=True)
# data_tip = data_tip.set_index('tip_num')

df_dict = {
    'bus_df': data_businesses,
    'checkin_df': data_checkin,
    'tip_df': data_tip,
    'review_df': data_review_sample,
    'user_df': data_user_sample
}


In [19]:

# Define the processing instructions
processes = {
    "bus_df": {
        "bool": ["is_open"],
        "dict": ["attributes", "hours"],
        "lists": ["categories"]
    },
    "checkin_df": {
        "date_time_list": ["date"]
    },
    "tip_df": {
        "date_time": ["date"],
        "string": ["text"]
    },
    "user_df": {
        "date_time": ["yelping_since"], #yelping since already shows in datetime...
        "lists": ["elite", "friends"]
    },
    "review_df": {
        "date_time": ["date"],
        "string": ["text"]
    }
}

for df_name, df in df_dict.items():
    
    instructions = processes[df_name]

    for dtype, cols in instructions.items():
        if dtype == "bool":
            for col in cols:
                df[col] = df[col].astype(bool)
        
        elif dtype == "date_time":
            for col in cols:
                df[col] = dd.to_datetime(df[col], errors='coerce')
        
        elif dtype == "date_time_list":
            for col in cols:
                df[col] = df[col].str.split(', ').map(
                    lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S", errors='coerce') 
                    if isinstance(x, list) else [],
                    meta=('x', 'object')
                )
        elif dtype == "dict":
            for col in cols:
                df[col] = df[col].map(
                    lambda x: ast.literal_eval(x) if pd.notna(x) and isinstance(x, str) else {}, 
                    meta=('x', 'object')
                )
        
        elif dtype == "lists":
            for col in cols:
                df[col] = df[col].map(
                    lambda x: x.split(', ') if pd.notna(x) and isinstance(x, str) else [], 
                    meta=('x', 'object')
                )
        
        elif dtype == "string":
            for col in cols:
                df[col] = df[col].str.lower()

    # df_dict[df_name] = df

# If you need to compute the results, you can do so here:
# for df_name in df_dict:
#     df_dict[df_name] = df_dict[df_name].compute()

In [22]:
data_checkin.compute().info()

<class 'pandas.core.frame.DataFrame'>
Index: 131930 entries, ---kPU91CF4Lq2-WlRu9Lw to zzyx5x0Z7xXWWvWnZFuxlQ
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   date    131930 non-null  object
dtypes: object(1)
memory usage: 4.8+ MB


In [29]:
data_user_sample.compute().info()

<class 'pandas.core.frame.DataFrame'>
Index: 4211 entries, --KsuCSkGGvDKTbdK9NvIg to zySfwbIHPNhLR8-JiqGdWA
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   name                4211 non-null   string        
 1   review_count        4211 non-null   int64         
 2   yelping_since       4211 non-null   datetime64[ns]
 3   useful              4211 non-null   int64         
 4   funny               4211 non-null   int64         
 5   cool                4211 non-null   int64         
 6   elite               4211 non-null   object        
 7   friends             4211 non-null   object        
 8   fans                4211 non-null   int64         
 9   average_stars       4211 non-null   float64       
 10  compliment_hot      4211 non-null   int64         
 11  compliment_more     4211 non-null   int64         
 12  compliment_profile  4211 non-null   int64         
 13  compliment_cut

In [31]:
data_user_sample.head(20)

,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
user_id,,,,,,,,,,,,,,,,,,,,,
--KsuCSkGGvDKTbdK9NvIg,C-Mac,57,2010-10-02 11:59:35,68,16,28,[],"[TcqGIB4Pbz6TtZu3PxLfVg, dDwwmnPyzSiNaD3yix_Ev...",5,3.93,...,0,1,0,0,2,5,8,8,5,2
--ccVMj2PN6Z9qtdOdlung,Kacie,15,2008-08-06 20:39:47,59,33,55,[],"[jCfyi8mOBqMJrj_DZFalgg, 0koQMZk1diY9f1gOv3nmP...",2,3.67,...,0,0,0,0,4,1,3,3,1,0
--qijw7qsKsG-v6rreDOAA,Amelia,23,2010-02-03 16:57:23,14,5,10,[],"[KQcfYKxkZIwLak-85Us7eA, bqjAhIGARynspNvUp2dQ3...",4,4.37,...,1,1,0,0,0,0,0,0,0,0
-4RbxLJlFZlu-KRuUiiGLw,Autumn,66,2008-09-13 04:37:14,240,50,130,[],"[OapLWhQI4_Gm32-nYbHmUA, NNgQ8fV5ARQgHw-Ob7l50...",8,4.13,...,2,0,1,0,3,6,19,19,0,5
-4swZSBw5Qdk5VRqB7kw1A,Tom,30,2011-02-13 21:47:31,67,4,6,[],"[A-ZQRbFI3xQbsyhzV0rRBg, JwV8ZSQz2T19eaDTcCUkd...",0,3.48,...,0,0,0,0,1,1,0,0,0,0
-5aeb8D7G7OHQ-Gvfl52MQ,Joshua,193,2006-01-26 18:01:18,249,75,117,[],"[ZIowdUve-IjIN-EgQIwCLA, XsV5D6lD8c9cRhfm-50x3...",6,4.27,...,3,0,0,0,6,15,4,4,1,0
-6GP9U1E374k1Pt4EyrNwA,Melanie,118,2011-07-27 02:15:20,27,5,5,[],"[TOF0VyF8SnwXMYHtkunELg, NkLqwtN7sNKuK5W4okbg9...",0,3.58,...,0,0,0,0,1,0,0,0,2,0
-6JgeM8-adaxpM_Mtr_ovw,Marceline,80,2008-04-08 21:58:46,127,15,43,[],"[Rd6NylyZCCU6CNRwc9G1Vw, NtjYwqmLBo29bmfRmoLHm...",4,3.51,...,2,1,0,0,7,3,7,7,3,0
-7q3rd8_NeMXnMGySFFpEg,Tara,303,2009-01-04 04:53:14,633,133,253,"[2009,2010,2011,2012,2013]","[fiGcLk0jd3nURd58oBx2lw, qkrfHp-DhK9Cp8rrdRZbX...",26,4.13,...,7,5,0,2,20,15,19,19,18,1
